In [1]:
import pandas as pd
import numpy as np

#### Load the data
We'll use Pandas to read the data from file

In [ ]:
df = pd.read_csv('data.csv')

#### Get to know your data set
We can start by exploring the various data types in the set

In [ ]:
df.info()


From our data, it is clear that our feature set most likely should consist of the column step, to newbalanceDest while the label column is isFraud
Let's create two variables holding the X and Y values for our model

In [ ]:
X = df[df.columns[:9]]
Y = df[df.columns[9:10]]


#### Transform the data
Tranforming the data includes multiple steps, e.g.:
- Replacing any missing values, infinity or NaN
- Transforming textual features
- Normalize the feature space

##### Explore missing values
Running the following command indicates that we do not have any missing values

In [ ]:
df.isna().sum()

##### Transform textual features
The type, nameOrig and nameDest are textual features which we need to transform to a shape the machine learning algorithm can understand.
The type column has finitive options, so we can do something called OneHotEncoding on the column to create binary columns.

Pandas provide a very useful function for this, get_dummies. We can utilize this function to create new binary columns, drop the original column and concatenate the two dataframes.

In [ ]:
X = pd.concat([X, pd.get_dummies(X['type'], prefix='type')], axis=1)

X.drop(['type'],axis=1, inplace=True)
X.head(10)

In [ ]:
X = pd.concat([X, pd.get_dummies(X['nameOrig'], prefix='nameOrig', sparse=True)], axis=1)
X.head(10)

#### Transforming nameOrig and nameDest
The following two columns to transform are nameOrig and nameDest
We can use a label encoder for this.

In [ ]:
from sklearn.preprocessing import LabelEncoder

categorical_feature_mask = X.dtypes==object
categorical_cols = X.columns[categorical_feature_mask].tolist()

le = LabelEncoder()

X[categorical_cols] = X[categorical_cols].apply(lambda col: le.fit_transform(col))
X.head(10)

#### Split the data
To evaluate the performance of the model, we will need a dataset for training and one for testing

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
X_train.shape

#### Train the model

In [ ]:
Y_train = Y_train.values.flatten()
Y_test = Y_test.values.flatten()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, class_weight='balanced')
clf.fit(X_train, Y_train)

#### Evaluate the model
Once the model has been trained, lets evaluate how accurate the model is

#### Accuracy

In [ ]:
metrics = clf.score(X_test, Y_test)

print(metrics)


Not bad, 99.9% accurate! But remember that the dataset was highly unbalanced (we can just guess non-fraudulent on all transactions and get 99.9% accuracy)
Let's look at other metrics

#### Precision-Recall

- **Precision**: High precision means that all transactions we flagged as fraudulent actually where (no false positives)
- **Recall**: High recall means that we did not miss any fraudulent transactions (no false negatives)

There is a trade off between both, lets see how we can illustrate that in a precision-recall curve

In [ ]:
from sklearn.metrics import classification_report
target_names = ['non-fraudulent', 'fraudulent']

pred = clf.predict(X_test)
print(classification_report(Y_test,pred, target_names=target_names))

Not bad! We can see that for fraudulent transactions, our recall is very high, but precision is low. This means that we create quite a few false positives but that is better than missing fraudulent activity

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

Y_score = clf.predict_proba(X_test)
Y_score = Y_score[:, 1]
Y_score = Y_score.flatten()

precisions, recalls, thresholds = precision_recall_curve(Y_test, Y_score)

plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
plt.plot(thresholds, recalls[:-1], "g--", label="Recall")
plt.legend(loc="center left")
plt.ylim([0, 1])
plt.xlim([0, 1])

plt.show()

#### Iterate
So our model is far from great, let's see if we can improve it. 
One of the issues we are seeing right now is that despite it's accuracy, it is missing a lot of fraudulent transactions.
A struggle we have is that our dataset is highly imbalanced. We can address this by weighing our inputs

Let's train our model again, but boosting our minority class, is fraudulent

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_with_weights = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, class_weight="balanced")
clf_with_weights.fit(X_train, Y_train)

In [ ]:
metrics = clf_with_weights.score(X_test, Y_test)

print(metrics)

Our accuarcy dropped, but let's see if we can outweight that with better recall-precision

In [ ]:
from sklearn.metrics import classification_report
target_names = ['non-fraudulent', 'fraudulent']

pred_with_weights = clf_with_weights.predict(X_test)
print(classification_report(Y_test, pred_with_weights, target_names=target_names))

Wow! We can see that especially recall greatly increased for the fraudulent cases, meaning that we miss a lot less false negatives

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

Y_score = clf_with_weights.predict_proba(X_test)
Y_score = Y_score[:, 1]
Y_score = Y_score.flatten()

precisions, recalls, thresholds = precision_recall_curve(Y_test, Y_score)

plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
plt.plot(thresholds, recalls[:-1], "g--", label="Recall")
plt.legend(loc="center left")
plt.ylim([0, 1])
plt.xlim([0, 1])

plt.show()